<a href="https://colab.research.google.com/github/sims-23/PySparkLearning/blob/main/pyspark_primer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, download Java. Next installing Apache Spark 3.3.0 with Hadoop 3.3 and then unzipping it.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz

Also need to install findspark which locates Spark on the system and imports it as a regular library. yay.

In [ ]:
!pip install -q findspark

Set up the environment path so we can run Pyspark in the Colab environment.

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

Locate Spark in the system (if want to find where the location is, findspark.find())

In [ ]:
import findspark
findspark.init()

SparkContext lets our application access a Spark cluster

In [ ]:
from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

In [ ]:
sc

<SparkContext master=local[*] appName=YourTest>

List of numbers is on the driver machine - driver because it's the machine that'll tell others to do. Spark will distribute data across multiple machines (or cluster of machines) which will do the processing for us. Cluster (distributed computing? - a group of machines dedicated to doing a specific task)

In [ ]:
nums = list(range(0, 1000001))
len(nums)

1000001

Distributing nums and gives back a type of RDD. RDD is ...

In [ ]:
nums_rdd = sc.parallelize(nums)
nums_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

Don't want to use collect function at this moment.



In [ ]:
nums_rdd.take(5)

[0, 1, 2, 3, 4]

Looking at applying a function to an RDD.

In [ ]:
squared_nums_rdd = nums_rdd.map(lambda x: x**2)
squared_nums_rdd.take(5)

pairs = squared_nums_rdd.map(lambda x: (x, len(str(x))))
pairs.take(5)

even_digits_pairs = pairs.filter(lambda x: x[1] % 2 == 0)
even_digits_pairs.take(5)

[(16, 2), (25, 2), (36, 2), (49, 2), (64, 2)]

For the last thing, wanted to cover groupbys for Spark. Spark has notion of key value pairs (key is what we're grouping on, value is what we're using for our computation) so first need to switch each element in list for our task of computing the average of the even digits squares with the same number of digits

In [ ]:
reverse_even_pairs = even_digits_pairs.map(lambda x: (x[1], x[0]))
reverse_even_pairs.take(5)

[(2, 16), (2, 25), (2, 36), (2, 49), (2, 64)]

In [ ]:
grouped = reverse_even_pairs.groupByKey()
grouped.take(5)

#seeing what the first two groups look like
visible_list_groups = grouped.map(lambda x: (x[0], list(x[1])))
visible_list_groups.take(2)

Looking to average the groups now. However, this was a slow way to average.

In [ ]:
averaged = grouped.map(lambda x: (x[0], sum(x[1])/len(x[1])))
averaged.collect()

[(2, 45.166666666666664),
 (4, 4675.5),
 (6, 471838.0),
 (8, 47204941.666666664),
 (10, 4720705565.0),
 (12, 472075391214.1667)]